# License Plate Detection on Moroccan Cars using YOLOv3 with Flow Normalizing (YoloFN v3)

![https://raw.githubusercontent.com/oublalkhalid/MoroccoAI-Data-Challenge/main/images/workflow.png](https://raw.githubusercontent.com/oublalkhalid/MoroccoAI-Data-Challenge/main/images/workflow.png)
This repository guides you through the steps for annotating and training a custom model to detect and blur the license plates on Morrocan licence plates.[Please consult our Pipline and the associated paper in the repository. ](http://github.com/oublalkhalid/MoroccoAI-Data-Challenge)
We also mention that more than 3go of data are generated to increase the training data of our Yolo (We also have the weight of yolo tiyni which gives a score of 0.59 on the dataset of this challenge).

> Please follow the associated article for more details - (Please check our paper after submission deadline).



# Step 1: Prepare the dataset

We get from Morroco-IA more than 18900 images of Morrocain licences with the number plate. 

1. To strat we create 2 folders 'test' and 'train' and transfer 20% of the images in test and 80% images in train folders respectively.

2. Alternatively, you can use the test and train data used by me for training. Download test.zip and train.zip files from our Github Repository for accessing the annotated images.

3. Annotate the license plates in all the images in the respective test and train folders, using LabelImg, and generate the annotated .xml file for the images within the respective folders. You can get more details on LabelImg from [here
](https://github.com/tzutalin/labelImg).(Conversion process is mentioned in step 3)


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
path="/kaggle/input"
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Step 2: Installing the base darknet system

In [ ]:
# Download YOLOv3 project
! git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Install the base darknet framework
%cd darknet
! make

In [ ]:
# Uplod test and train directories in the darknet directory along with the convert.py file from this respository and run the below command
# This step will pick the coordinates from each .xml file and put them into YOLO compatible .txt file in the same test and train directories.
 # Also, train.txt and test.txt files are created within the darknet folder, containing the location of images.

#python convert.py
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join


def get_images_in_dir(dir_path):
    image_list = []
    for filename in glob.glob(dir_path + '/*.jpg'):
        image_list.append(filename)

    return image_list


def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)


def convert_annotation(dir_path, output_path, image_path):
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]

    in_file = open(dir_path + '/' + basename_no_ext + '.xml')
    out_file = open(output_path + basename_no_ext + '.txt', 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')


cwd = getcwd()
dirs = ['train','test']
classes = ['LP']

for dir_path in dirs:
    full_dir_path = cwd + '/' + dir_path
    output_path = full_dir_path + '/'

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    image_paths = get_images_in_dir(full_dir_path)
    list_file = open(full_dir_path + '.txt', 'w')

    for image_path in image_paths:        
        list_file.write(image_path + '\n')
        convert_annotation(full_dir_path, output_path, image_path)
    list_file.close()

    print("Finished processing: " + dir_path)

# Analysis Input Data

In [ ]:
import pandas as pd
import sys
import matplotlib.pyplot as plt
from tqdm import trange 
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
train_string=pd.read_csv(path+'/moroccoai-data-challenge-edition-001/train.csv')
train_string["class"]=train_string["plate_string"]
train_string.astype(str)
train_string.head(15)

In [ ]:
for k in trange((train_string.shape[0])):
    if "b" in train_string["plate_string"][k]:
        train_string["class"][k]="b"
    elif "j" in train_string["plate_string"][k]:
        train_string["class"][k]="j"
    elif "ch" in train_string["plate_string"][k]:
        train_string["class"][k]="ch"
    elif "ww" in train_string["plate_string"][k]:
        train_string["class"][k]="w"
    elif "m" in train_string["plate_string"][k]:
        train_string["class"][k]="m"
    elif "a" in train_string["plate_string"][k] and "waw" not in train_string["plate_string"][k]:
        train_string["class"][k]="a"
    elif "d" in train_string["plate_string"][k]:
        train_string["class"][k]="d"
    elif "h" in train_string["plate_string"][k] and "ch" not in train_string["plate_string"][k] :
        train_string["class"][k]="h"
    elif "waw" in train_string["plate_string"][k]:
        train_string["class"][k]="ch"
    else: 
        train_string["class"][k]="other"
train_string.head(10)

## We have Unbalanced case on the training Dataset --> Log Transformation

In [ ]:
train_string.groupby(['class']).sum().unstack().plot(kind='bar',color='green')
plt.xlabel("Class")
plt.ylabel("Count")

# Step 4: Prepare custom 'my_data' folder

In [ ]:
# Run the below command to make a custom folder named 'my_data'
! mkdir my_data

In [ ]:
# Move the train.txt and test.txt files from the darknet directory to the my_data directory.
! mv train.txt my_data/ 
! mv test.txt my_data/

In [ ]:
# Create classes.names file within my_data directory with class name as "LP"
! touch /content/gdrive/MyDrive/darknet/my_data/classes.names 
! echo LP > /content/gdrive/MyDrive/darknet/my_data/classes.names

In [ ]:
# Create weights directory within the my_data directory
! mkdir my_data/weights

In [ ]:
# Create file darknet.data within my_data directory to provide the configuration details
! touch /content/gdrive/MyDrive/darknet/my_data/darknet.data

# Paste the below details manually in darknet.data file
## classes = 1
## train = my_data/train.txt
## valid = my_data/test.txt
## names = my_data/classes.names
## backup = my_data/weights/

In [ ]:
# Copy and Paste the cfg file from darknet/cfg/yolov3.cfg to darknet/my_data directory
! cp /content/gdrive/MyDrive/darknet/cfg/yolov3.cfg /content/gdrive/MyDrive/darknet/my_data

# Make the following changes in yolov3.cfg in my_data directory.
# Line 603, 693, and 780 change the filters to 18. (filters = (classes + 5) * 3). 
#In our case we are detecting only 1 class, so the number of filters will be equal to 18.
# Line 783, change the number of classes to 1.

# **Step 5: Download the initial yolo weights for training the custom data**

In [ ]:
# Run the following command from the darknet directory
! wget https://pjreddie.com/media/files/darknet53.conv.74

## Step 6: Set criteria to save the weights file in the weights **directory**

In [ ]:
# Open detector.c file from darknet/examples directory and change line number 138 as shown below.

if(i%1000==0 or (i < 1000 and i%200 == 0)):

# This change saves the weight in my_data/weights directory for every 200th iteration till 1000 iterations and then for every 1000th iteration.

## Step 7: Now start the **training**

In [ ]:
# Run the following command from the darknet directory
! ./darknet detector train /content/gdrive/MyDrive/darknet/my_data/darknet.data /content/gdrive/MyDrive/darknet/my_data/yolov3.cfg /content/gdrive/MyDrive/darknet/darknet53.conv.74

## Step 8: Detect and Blur the Licence Plate from the **image**

In [ ]:
# Change the weightsPath in line number 12 and the image file name in line number 15 before running the below code

%matplotlib inline
import numpy as np
import imutils
import cv2
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

CONF_THRESH, NMS_THRESH = 0.5, 0.5


weightsPath = 'custom/weights/yolov3_8000.weights'
configPath = 'custom/yolov3.cfg'
namesPath = 'custom/classes.names'
image = 'img3.jpeg'

# Load the network using openCV
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Get the output layer from YOLOv3
layers = net.getLayerNames()
output_layers = [layers[i[0] - 1] for i in net.getUnconnectedOutLayers()]


# Read and convert the image to blob and perform forward pass
img = cv2.imread(image)
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
height, width = img.shape[:2]

blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
layer_outputs = net.forward(output_layers)


class_ids, confidences, b_boxes = [], [], []
for output in layer_outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > CONF_THRESH:
            center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            b_boxes.append([x, y, int(w), int(h)])
            confidences.append(float(confidence))
            class_ids.append(int(class_id))


# Perform non maximum suppression for the bounding boxes to filter overlapping and low confident bounding boxes
indices = cv2.dnn.NMSBoxes(b_boxes, confidences, CONF_THRESH, NMS_THRESH).flatten().tolist()

if len(indices) > 0:

    # Draw the filtered bounding boxes with their class to the image
    with open(namesPath, "r") as f:
        classes = [line.strip() for line in f.readlines()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))

    for index in indices:
        (x,y) = (b_boxes[index][0], b_boxes[index][1])
        (w,h) = (b_boxes[index][2], b_boxes[index][3])
        
        # Blur the ROI of the detected licence plate 
        img[y:y+h, x:x+w] = cv2.GaussianBlur(img[y:y+h, x:x+w] ,(35,35),0)

        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text = "{}: {:.4f}".format("LP", confidences[index])
        cv2.putText(img, text, (x, y - 3), cv2.FONT_HERSHEY_COMPLEX_SMALL, .75 , (0, 255, 0), 1)

plt.figure(figsize=(10, 5))
plt.imshow(img)
plt.show()